# Extract - Transform - Load
Dit script leest alle top2000 data en importeert deze in de database.

In [1]:
from itertools import islice
import copy # for deepcopy
import Levenshtein


# Definities

## Data objects
Definieren van alle Entities: Occurance, Song en Artist

In [2]:
class Occurance:
    """
    Represents exactly one row from one T2k edition
    """
    TITLE_INDEX = 0
    ARTIST_INDEX = 1
    YEAR_INDEX = 2
    EDITION_INDEX = 3
    POSITION_INDEX = 4
    
    
    def __init__(self, row):
        self.data = row
        self.key = self.__generate_key__(row)
        self.artist_key = self.__normasize__(row[self.ARTIST_INDEX])
        self.edition = row[self.EDITION_INDEX]
        self.position = row[self.POSITION_INDEX]
        
    def is_from_year(self, year):
        """
        Returns True if and only if we are sure that this song is published in the given year
        """
        return self.data[self.YEAR_INDEX] is not None and self.data[self.YEAR_INDEX]==year
    
    
    def __generate_key__(self, row):
        """
        Returns a string representing the normasized title and artist
        """
        return self.__normasize__(row[self.TITLE_INDEX], row[self.ARTIST_INDEX])

    
    def __normasize__(self, *args):
        """
        Returns a string representing a normalized text specific for song titles and artist names.
        """
        # first, join all arguments into one single string, where all None args are removed
        s = " ".join([str(s) for s in list(args) if s is not None])
        s = s.lower() # change to lower case
        if s.startswith("the "): # remove leading the
            s = s[4:]
        if s.startswith("de "): # remove leading de
            s = s[3:]
        if s.startswith("les "): # remove leading de
            s = s[4:]
        # remove some other standard words
        s = (s.replace("/", " ") 
                .replace("-", " ")
                .replace("'", "")
                .replace(".", "")
                .replace(",", "")
                .replace("!", "")
                .replace("?", "")
                .replace("(", "")
                .replace(")", "")
                .replace("ë", "e")
                .replace("ö", "o")
                .replace("å", "a")
                .replace("ø", "o")
                .replace(" the ", " ") # should handle artist 'the the' nicely
                .replace(" de ", " ")
                .replace(" and ", " ")
                .replace(" en ", " ")
                .replace(" & ", " ")
                .replace(" feat", "")
                .replace(" ft", "")
                .replace("\xa0", " ")
                .replace("\ufeff", "")
                .replace(" albumversie ", " ")
        )
        while "  " in s: # remove all extra spaces
            s = s.replace("  ", " ")
        s = s.strip() # remove all leading and trailing spaces
        return s
    
    def __repr__(self):
        return "{} - {} ({}): {}".format(
            self.data[self.ARTIST_INDEX], self.data[self.TITLE_INDEX], self.data[self.YEAR_INDEX], 
            self.data[self.POSITION_INDEX])    

In [3]:
class Song:
    """
    Handles all about one song, including all its T2k occurances, key generation
    """
    
    def __init__(self, occurance):
        """
        Constructor
        """
        self.occurances = []
        self.keys = []
        self.add_occurance(occurance)

        
    def is_exact_match(self, occurance):
        """
        returns True is one of the keys is an exact match of the occurance's key
        """
        return self.has_key(occurance.key)
        
        
    def has_key(self, key):
        """
        returns True is one of the keys is an exact match of the specified key
        """
        return key in self.keys
        
        
    def add_occurance(self, occurance):
        """
        Adds an occurance to this song
        """
        self.occurances.append(occurance)
        if occurance.key not in self.keys:
            self.keys.append(occurance.key)
        self.key = occurance.key
        self.title = occurance.data[Occurance.TITLE_INDEX]
        self.artist = occurance.data[Occurance.ARTIST_INDEX]
        if occurance.data[Occurance.YEAR_INDEX] is not None: 
            # remember previous year if year is not filled in
            self.year = occurance.data[Occurance.YEAR_INDEX]
    
    
    def get_best_match(self, occurance):
        best = 0
        for occ in self.occurances:
            ratio = Levenshtein.ratio(occurance.key, occ.key)
            if ratio > best:
                best = ratio
        return best
        
    def get_best_key(self):
        """
        Returns the best available key from the occurances
        """
        return self.key
    
    def get_artist_keyset(self):
        result = []
        for occ in self.occurances:
            result.append(occ.artist_key)
        return set(result)
        

    def __repr__(self):
        return "{} - {} ({})".format(self.artist, self.title, self.year)

In [4]:
class Artist:
    """
    Hanldes all about one artist, including all its songs, key generation
    """
    
    def __init__(self, song):
        """
        Constructor 
        """
        self.songs = []
        self.keys = song.get_artist_keyset()
        self.add_song(song)

        
    def is_exact_match(self, song):
        """
        returns True is one of the keys is an exact match of one of the the song's artist_keyset entries
        """
        for k in song.get_artist_keyset():
            if self.has_key(k):
                return True
        return False
        
        
    def has_key(self, key):
        """
        returns True is one of the keys is an exact match of the specified key
        """
        return key in self.keys
        
        
    def add_song(self, song):
        """
        Adds a song to this artist
        """
        self.songs.append(song)
        self.name = song.artist
        for k in song.get_artist_keyset():
            self.keys.add(k)
    
    def __repr__(self):
        return "{} ({} liedjes)".format(self.name, len(self.songs))

## Editieprocessor
De editieprocessor klasse is bedoelt om de data van één T2k editie te importeren. 

Het proces is als volgt:

1. `new_entries`: maakt een nieuwe Song aan voor alle Occurances van liedjes die zijn uitgebracht in het jaar van de editie
1. `exact_match`: koppelt een Occurance aan een Song als één van de keys van de Song een exacte match (occ.key in song.keys) is
1. `manual_match`: een Occurance wordt gekoppeld aan een Song door een hardgecodeerde lijst
1. `direct_similarity`: koppel, op volgorde van _best passendheid_ de _best passende_ Song bij een occurance totdat de best passende kleiner is dan een ingestelde limiet. Deze limiet bepaalt dus vanaf waar we niet zeker zijn van een passende koppeling
1. `print_uncertain_matches`: (als er nog Occurances zijn die een similarity hebben groter dan de ingestelde minimaal relevante similarity) print makkelijk te kopieren info over deze kandidaat-koppelingen voor de manual_matchlist
1. `add_remaining_occurances_as_new_songs`: (als hierboven niet waar is) zoals de titel al zegt


In [5]:

class EditionProcessor():
    """
    One Object can handle only One Edition Once
    """
    
    direct_match_count = 0
    songs_from_edition_count = 0
    similarity_match_count = 0
    new_songs_from_similarity_matrix = 0
    matches_from_similarity_matrix = 0
    
    def __init__(self, edition_reader=[], existing_data=[]
                , min_certain_ratio=0.9, min_relevant_ratio=0.6):
        self.new_song_list = []
        self.matched_song_occurance_tuple_list = []
        self.problem_tuple_list = []
        self.smtrx = {}
        self.min_certain_ratio = min_certain_ratio
        self.min_relevant_ratio = min_relevant_ratio
        self.year = edition_reader.edition
        self.picklist = [song for song in existing_data] # list of keys that ar not yet matched in this edition
        self.edition_dict = {}
        self.edition_data = [Occurance(row) for row in edition_reader]
        print("{} liedjes in de editie".format(len(self.edition_data)))
        

    def register_as_problem(self, problem_tuple):
        """
        Registers an occurance as a new song
        """
        self.problem_tuple_list.append(problem_tuple)

        
    def register_as_new_song(self, occurance):
        """
        Registers an occurance as a new song
        """
        self.new_song_list.append(Song(occurance))

        
    def register_as_match(self, song, occurance):
        """
        Registers an occurance as a match to a song
        """
        if song in self.picklist:
            self.picklist.remove(song)
        else:
            print("{} not in picklist".format(song))
        self.matched_song_occurance_tuple_list.append((song, occurance))
    
    def process_new_songs(self):
        """
        Adds all occurances as new songs if the year of the song is equal to the edition's year
        """
        self.songs_from_edition_count = 0
        remaining_edition_data = []
        for occ in self.edition_data:
            if occ.is_from_year(self.year):
                self.register_as_new_song(occ)                
                self.songs_from_edition_count+=1
            else:
                remaining_edition_data.append(occ)
        self.edition_data = remaining_edition_data
        print("{:5d} liedjes die in {} zijn uitgebracht".format(self.songs_from_edition_count, self.year))
        
    
    def print_multiple_matches(self, occ, matches):
        print("Ik heb een liedje met meer dan één match gevonden voor {}".format(occ))
        for m in matches:
            print(m)
    
    def process_exact_matches(self):
        """
        moves rows from edition_data only if the hash of the row exactly matches a key in the t2000_data
        """
        self.direct_match_count = 0
        remaining_edition_data = []
        for occ in self.edition_data:
            exact_matches = []
            for song in self.picklist:
                if song.is_exact_match(occ):
                    exact_matches.append(song)
            if len(exact_matches) > 0:
                if len(exact_matches) > 1:
                    self.print_multiple_matches(occ, exact_matches)
                else: # is exact 1
                    self.register_as_match(exact_matches[0], occ)
                    self.direct_match_count += 1
            else:
                remaining_edition_data.append(occ)
        self.edition_data = remaining_edition_data        
        print("{:5d} liedjes exact gematched".format(self.direct_match_count))
                
    
    def find_song_from_picklist_by_key(self, key):
        """
        Finds and returns the first song from the picklist where the specified key exactly matches one of its keys. 
        Otherwise it returns None
        """
        for song in self.picklist:
            if song.has_key(key):
                return song
        return None
    
    def process_manual_matches(self, match_dict={}):
        """
        Adds occurances to songs using the specified dictionary: { occurance_key : song_key }
        """
        self.manual_match_count = 0
        remaining_edition_data = []
        for occ in self.edition_data:
            if occ.key in match_dict:
                song = self.find_song_from_picklist_by_key(match_dict[occ.key])
                self.register_as_match(song, occ)
                self.manual_match_count += 1
            else:
                remaining_edition_data.append(occ)
        self.edition_data = remaining_edition_data
        print("{:5d} liedjes handmatig gematched".format(self.manual_match_count))
        
    
    def build_similarity_matrix(self):
        """
        Builds the similarity matrix: {occ : { ratio : song}}
        """
        print("Building similarity matrix...")
        remaining_edition_data = []
        self.smtrx = {}
        for occ in self.edition_data:
            data = {}
            for song in self.picklist:
                ratio = song.get_best_match(occ)
                if ratio >= self.min_relevant_ratio:
                    data[ratio] = song
            if len(data) > 0:
                self.smtrx[occ] = data
            else: # occurance cannot be matched, so add to remaining data
                remaining_edition_data.append(occ)
        self.edition_data = remaining_edition_data
        print("Done")
                

    def pop_largest_similarity(self):
        """
        Pops a tuple: (song, occurance, ratio) of the best possible match currently available in the similarity matrix
        and returns it. 
        If no match is found, because the similarity matrix is empty, it returns None. 
        When an occurance has no candidate song matches it returns a tuple (None, occurance, -1)
        """
        if len(self.smtrx)==0: # return None is matrix is empty
            return None
        
        best_ratio = -1
        best_occ = None
        best_song = None
        # find the highest ratio, occurance and song
        for occ, songs in self.smtrx.items():
            if len(songs)==0 and best_occ is None: # if we haven't found a bast yet and we encounter an empty dict
                best_occ = occ
            for ratio, song in songs.items():
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_occ = occ
                    best_song = song

        # remove the found occurance from the similarity matrix
        del self.smtrx[best_occ]
        
        if best_song is not None:
            # remove the found song from all dicts in the similarity matrix
            for songs in self.smtrx.values():
                # Using dictionary comprehension to find list 
                delete = [key for key in songs if songs[key] == best_song]
                for key in delete: del songs[key]            

        return (best_song, best_occ, best_ratio)
        
        
    def process_near_matches(self):
        """
        Processes the tuples in the similarity matrix. It either adds the tuple as a match, a problem (pending on the
        min_certain_ratio) or a new song (if pop_largest_similarity returns (None, occurance, -1)).
        """
        self.build_similarity_matrix()
        self.matches_from_similarity_matrix = 0
        while True:
            # match_tuple is: (occurance, song, similarity)
            match_tuple = self.pop_largest_similarity()

            if match_tuple is None: break # break the loop if there was nothin to pop left
            
            if match_tuple[0] is None: # if there was no song to match
                self.register_as_new_song(match_tuple[1])
            elif match_tuple[2] < self.min_certain_ratio: # if the ratio was lower than the minimum relevant threshold
                self.register_as_problem(match_tuple)
            else:
                self.register_as_match(match_tuple[0], match_tuple[1])
                self.matches_from_similarity_matrix += 1
        print("{:5d} liedjes gematched vanuit de similarity matrix".format(self.matches_from_similarity_matrix))
                
        
    def process_rest_as_new(self):
        """
        Processes the remander of edition_data as new songs
        """
        count = 0
        for occ in self.edition_data:
            self.register_as_new_song(occ)
            count+=1
        self.edition_data = []
        print("{:5d} liedjes niet gematched dus als nieuw toegevoegd".format(count))
            
            
    def has_problems(self):
        """
        Returns True if the processor has problems registered that avoid a correct merge
        """
        return len(self.problem_tuple_list) > 0
    
    
    def merge(self, existing_data):
        """
        Merges the imported data into the specified existing data
        """
        # add new songs
        for new in self.new_song_list:
            existing_data.append(new)
        
        # update matches
        for song, occ in self.matched_song_occurance_tuple_list:
            song.add_occurance(occ)

        return existing_data
    
    
    def print_status(self):
        pl = len(self.picklist)
        ed = len(self.edition_data)
        nw = len(self.new_song_list)
        mt = len(self.matched_song_occurance_tuple_list)
        sx = len(self.smtrx)
        pb = len(self.problem_tuple_list)
        print("Edition_data: {:4d} + {:4d} + {:4d} + {:4d} + {:4d} = {:4d}".format(
            ed, nw, mt, sx, pb, ed+nw+mt+sx+pb))
        print("Picklist    : {:4d} + {:4d} = {:4d}".format(pl, mt, pl+mt))
        
        
    def print_problems(self):
        print()
        print("=== {} PROBLEEMGEVALLEN ===".format(len(self.problem_tuple_list)))
        for item in self.problem_tuple_list:
            print("    '{}' : '{}', #{}".format(item[1].key, item[0].key, item[2]))        
        

# Extract & Transform
Dit deel leest alle bronbestanden, en extraheert songs met behulp van de EditionProcessor.

In [6]:
t2k_data = []

## Edities 1999 - 2014
Deze edities staan in één groot bestand, en worden dus tegelijk gelezen.

In [7]:
from reader.LijstenAlleJarenReader import LijstenAlleJarenReader

t2k_1999_2014_data = {} # dict that holds all Songs from 1999-2014 editions    
for row in LijstenAlleJarenReader():
    if row[4] != 0: # ignore iterations that return a zero position
        occ = Occurance(row)
        if not occ.key in t2k_1999_2014_data:
            t2k_1999_2014_data[occ.key] = Song(occ)
        else:
            t2k_1999_2014_data[occ.key].add_occurance(occ)

        
# t2k_data holds all extracted data
t2k_data = list(t2k_1999_2014_data.values())
print ("%d items" % len(t2k_data))

Reading raw/lijsten_alle_jaren.csv...
3906 items


## Edities 2015 en verder
Deze worden per editie verwerkt.

In [8]:
from reader.CSVReader import CSVReader
from reader.XlrdReader import XlrdReader
from reader.OpenPyXlReader import OpenPyXlReader

In [9]:
reader = CSVReader('raw/TOP-2000-2015.csv', 2015)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.82,
                            min_relevant_ratio=0.81)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'stay faraway so close u2' : 'stay so close far away u2', #0.6530612244897959
    'maybe im amazed paul mccartney wings' : 'maybe im amazed wings', #0.7368421052631579
    'bonfire heart james blunt' : 'bonfire heart blunt james', #0.76
    'rather be clean bandit jess glynne' : 'rather be clean bandit', #0.7857142857142857
    'valerie mark ronson amy winehouse' : 'valerie amy winehouse', #0.7777777777777778
    'where wild roses grow nick cave kylie minogue' : 'where wild roses grow kylie minogue nick cave', #0.7777777777777778
    'stay with me till morning vicky brown' : 'stay with me till morning vicki brown new london chorale', #0.7741935483870968
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)

Reading raw/TOP-2000-2015.csv...
2000 liedjes in de editie
   29 liedjes die in 2015 zijn uitgebracht
 1779 liedjes exact gematched
    7 liedjes handmatig gematched
Building similarity matrix...
Done
   23 liedjes gematched vanuit de similarity matrix
  162 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +  191 + 1809 +    0 +    0 = 2000
Picklist    : 2097 + 1809 = 3906


In [10]:
reader = XlrdReader('raw/TOP-2000-2016.xls', 2016)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.813,
                            min_relevant_ratio=0.81)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'voodoo chile jimi hendrix experience' : 'voodoo chile slight return jimi hendrix', #0.6666666666666666
    'single ladies beyoncé' : 'single ladies put a ring on it beyoncé', #0.711864406779661
    'barcelona freddie mercury montserrat caballé' : 'barcelona freddie mercury', #0.7246376811594203
    'eve of war jeff wayne' : 'eve of war jeff wayne justin hayward', #0.7368421052631579
    'purple rain prince revolution' : 'purple rain prince', #0.7659574468085106
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)


Reading raw/TOP-2000-2016.xls...
2000 liedjes in de editie
   19 liedjes die in 2016 zijn uitgebracht
 1797 liedjes exact gematched
    5 liedjes handmatig gematched
Building similarity matrix...
Done
   32 liedjes gematched vanuit de similarity matrix
  147 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +  166 + 1834 +    0 +    0 = 2000
Picklist    : 2263 + 1834 = 4097


In [11]:
reader = OpenPyXlReader('raw/TOP-2000-2017.xlsx', 2017, start_row=2)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.805,
                            min_relevant_ratio=0.801)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'raspberry beret prince revolution' : 'raspberry beret prince', #0.8
    'i would die 4 u prince revolution' : 'i would die 4 u prince', #0.8
    'when doves cry prince revolution' : 'when doves cry prince', #0.7924528301886793
    'liquid spirit claptone remix gregory porter' : 'liquid spirit gregory porter', #0.7887323943661971
    'dont leave me this way communards sarah jane morris' : 'dont leave me this way communards', #0.7857142857142857
    'forbidden colours ryuichi sakamoto david sylvian' : 'forbidden colours david sylvian', #0.7848101265822784
    'lets go crazy prince revolution' : 'lets go crazy prince', #0.7843137254901961
    'catch release deepend remix matt simons' : 'catch release matt simons', #0.78125
    'id do anything for love but i wont do that meat loaf' : 'id do anything for love meat loaf', #0.7764705882352941
    'overture tommy who' : 'ouverture who', #0.7741935483870968
    'orinoco flow sail away enya' : 'orinoco flow enya', #0.7727272727272727
    'senza una donna zucchero paul young' : 'senza una donna zucchero fornaciari', #0.7714285714285715
    'never be clever herman brood his wild romance' : 'never be clever herman brood', #0.7671232876712328
    'you dont bring me flowers neil diamond barbra streisand' : 'you dont bring me flowers barbra streisand neil diamond', #0.7636363636363637
    'tom trauberts blues waltzing mathilda tom waits' : 'tom trauberts blues tom waits', #0.7631578947368421
    'dont let me down beatles' : 'love me do beatles', #0.7619047619047619
    'saturday night herman brood his wild romance' : 'saturday night herman brood', #0.7605633802816901
    'laat me vivre alderliefste ramses shaffy liesbeth list' : 'laat me vivre ramses shaffy liesbeth list alderliefste', #0.7592592592592593
    'sylvias mother dr hook medicine show' : 'sylvias mother dr hook', #0.7586206896551724
    '32 jaar sinds 1 dag of 2 doe maar' : 'sinds 1 dag of 2 32 jaar doe maar', #0.7575757575757576
    'wake me up avicii aloe blacc' : 'wake me up avicii', #0.7555555555555555
    'diamante zucchero' : 'diamante zucchero fornaciari', #0.7555555555555555
    'still believe herman brood his wild romance' : 'still believe herman brood', #0.7536231884057971
    'there must be an angel playing with my heart eurythmics' : 'there must be an angel eurythmics', #0.75
    'everybody backstreets back backstreet boys' : 'everybody backstreet boys', #0.746268656716418
    'il volo zucchero' : 'il volo zucchero fornaciari', #0.7441860465116279
    'gotta catch m all pokemon theme song jason paige' : 'gotta catch em all jason paige', #0.7435897435897436
    'its end of world as we know it rem' : 'its end of world rem', #0.7407407407407407
    'sailing to philadelphia mark knopfler james taylor' : 'sailing to philadelphia james taylor mark knopfler', #0.74
    'folsom prison blues johnny cash' : 'folsom prison blues live in folsom prison johnny cash', #0.7380952380952381
    'sweet dreams are made of this eurythmics' : 'sweet dreams eurythmics', #0.7301587301587301
    'road ahead miles of unknown city to city' : 'road ahead city to city', #0.7301587301587301
    'against all odds take a look at me now phil collins' : 'against all odds phil collins', #0.725
    'mad world michael andrews gary jules' : 'mad world gary jules', #0.7142857142857143
    'rhiannon will you ever win fleetwood mac' : 'rhiannon fleetwood mac', #0.7096774193548387
    'refugee tom petty heartbreakers' : 'refugee tom petty', #0.7083333333333334
    'san quentin johnny cash' : 'san quentin live at san quentin johnny cash', #0.696969696969697
    'norwegian wood beatles' : 'norwegian wood this bird has flown beatles', #0.6875
    'get lucky daft punk pharrell williams' : 'get lucky daft punk', #0.6785714285714286
    'diamonds pearls prince new power generation' : 'diamonds pearls prince', #0.676923076923077
    'une belle histoire een mooi verhaal alderliefste paul leeuw' : 'une belle histoire paul leeuw alderliefste', #0.6732673267326733
    'kiss prince revolution' : 'kiss prince', #0.6666666666666666
    'la song beth hart' : 'la song out of this town beth hart', #0.6666666666666666
    'stuck in a moment you cant get out of u2' : 'stuck in a moment u2', #0.6666666666666666
    'tonight live david bowie tina turner' : 'tonight tina turner david bowie', #0.6567164179104478
    'droomland andré hazes paul leeuw' : 'droomland paul leeuw andré hazes', #0.65625
    'think blues brothers version aretha franklin' : 'think aretha franklin', #0.6461538461538462
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)


Reading raw/TOP-2000-2017.xlsx...
2000 liedjes in de editie
   20 liedjes die in 2017 zijn uitgebracht
 1797 liedjes exact gematched
   47 liedjes handmatig gematched
Building similarity matrix...
Done
   93 liedjes gematched vanuit de similarity matrix
   43 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +   63 + 1937 +    0 +    0 = 2000
Picklist    : 2326 + 1937 = 4263


In [12]:
reader = OpenPyXlReader('raw/TOP-2000-2018.xlsx', 2018, start_row=1)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.8,
                            min_relevant_ratio=0.8)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'cose della vita eros ramazzotti tina turner' : 'cose della vita tina turner eros ramazzotti', #0.7209302325581395
    'fourfiveseconds rihanna paul mccartney kanye west' : 'fourfiveseconds rihannna kanye west paul mccartney', #0.7676767676767676
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)

Reading raw/TOP-2000-2018.xlsx...
2000 liedjes in de editie
    5 liedjes die in 2018 zijn uitgebracht
 1882 liedjes exact gematched
    2 liedjes handmatig gematched
Building similarity matrix...
Done
    5 liedjes gematched vanuit de similarity matrix
  106 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +  111 + 1889 +    0 +    0 = 2000
Picklist    : 2437 + 1889 = 4326


In [13]:
reader = OpenPyXlReader('raw/TOP-2000-2019.xlsx', 2019, start_row=2)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.792,
                            min_relevant_ratio=0.785)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'forever autumn jeff wayne' : 'forever autumn jeff wayne justin hayward', #0.7692307692307693
    'treur niet ode aan het leven diggy dex jw roy' : 'treur niet diggy dex jw roy', #0.75
    'islands in stream kenny rogers dolly parton' : 'islands in stream dolly parton kenny rogers', #0.6976744186046512
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)

Reading raw/TOP-2000-2019.xlsx...
2000 liedjes in de editie
   14 liedjes die in 2019 zijn uitgebracht
 1891 liedjes exact gematched
    3 liedjes handmatig gematched
Building similarity matrix...
Done
   33 liedjes gematched vanuit de similarity matrix
   58 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +   73 + 1927 +    0 +    0 = 2000
Picklist    : 2510 + 1927 = 4437


In [14]:
reader = OpenPyXlReader('raw/TOP-2000-2020.xlsx', 2020, start_row=2, title_col=3, artist_col=2)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.89,
                            min_relevant_ratio=0.85)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'woman in chains tears for fears oleta adams' : 'woman in chains tears for fears', #0.8378378378378378
    'new gold dream 81 82 83 84 simple minds' : 'new gold dream simple minds', #0.8181818181818182
    'love me just a little bit more totally hooked on you dolly dots' : 'love me just a little bit more dolly dots', #0.7884615384615384
    'ghost love score live@wacken 2013 nightwish' : 'ghost love score nightwish', #0.7536231884057971
    'vluchten kan niet meer jenny arean frans halsema' : 'vluchten kan niet meer frans halsema jenny arean', #0.75
    'just two of us bill withers grover washington jr' : 'just two of us grover washington jr bill withers', #0.7291666666666666
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)

Reading raw/TOP-2000-2020.xlsx...
2000 liedjes in de editie
    8 liedjes die in 2020 zijn uitgebracht
 1908 liedjes exact gematched
    6 liedjes handmatig gematched
Building similarity matrix...
Done
   15 liedjes gematched vanuit de similarity matrix
   63 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +   71 + 1929 +    0 +    0 = 2000
Picklist    : 2581 + 1929 = 4510


In [15]:
reader = OpenPyXlReader('raw/TOP-2000-2021.xlsx', 2021, start_row=3)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.83,
                            min_relevant_ratio=0.83)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)

Reading raw/TOP-2000-2021.xlsx...
2000 liedjes in de editie
   12 liedjes die in 2021 zijn uitgebracht
 1928 liedjes exact gematched
    0 liedjes handmatig gematched
Building similarity matrix...
Done
    8 liedjes gematched vanuit de similarity matrix
   52 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +   64 + 1936 +    0 +    0 = 2000
Picklist    : 2645 + 1936 = 4581


In [16]:
reader = OpenPyXlReader('raw/NPORadio2Top2000-2022.xlsx', 2022, start_row=3)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.83,
                            min_relevant_ratio=0.79)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'hey joe jimi hendrix' : 'hey joe jimi hendrix experience', #0.7843137254901961
    'leave door open silk sonic bruno mars anderson paak' : 'leave door open silk sonic', #0.6753246753246753
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)

Reading raw/NPORadio2Top2000-2022.xlsx...
2000 liedjes in de editie
   12 liedjes die in 2022 zijn uitgebracht
 1898 liedjes exact gematched
    2 liedjes handmatig gematched
Building similarity matrix...
Done
   16 liedjes gematched vanuit de similarity matrix
   72 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +   84 + 1916 +    0 +    0 = 2000
Picklist    : 2729 + 1916 = 4645


In [17]:
reader = OpenPyXlReader('raw/Top-2000-2023.xlsx', 2023, start_row=2)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.76,
                            min_relevant_ratio=0.76)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'love story taylors version taylor swift' : 'love story taylor swift', #0.7419354838709677
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)

Reading raw/Top-2000-2023.xlsx...
2000 liedjes in de editie
   13 liedjes die in 2023 zijn uitgebracht
 1919 liedjes exact gematched
    1 liedjes handmatig gematched
Building similarity matrix...
Done
   11 liedjes gematched vanuit de similarity matrix
   56 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +   69 + 1931 +    0 +    0 = 2000
Picklist    : 2798 + 1931 = 4729


In [18]:
reader = OpenPyXlReader('raw/NPORadio2-Top-2000-2024.xlsx', 2024, start_row=2)

processor = EditionProcessor(edition_reader=reader, 
                            existing_data=t2k_data,
                            min_certain_ratio=0.79,
                            min_relevant_ratio=0.79)

processor.process_new_songs()
processor.process_exact_matches()
processor.process_manual_matches({
    'maneater daryl hall john oates' : 'maneater hall oates', #0.7755102040816326
    'shake it off taylors version taylor swift' : 'shake it off taylor swift', #0.7575757575757576
    'blank space taylors version taylor swift' : 'blank space taylor swift', #0.75
})
processor.process_near_matches()
processor.process_rest_as_new()
processor.print_status()
if processor.has_problems():
    processor.print_problems()
else:
    t2k_data = processor.merge(t2k_data)

Reading raw/NPORadio2-Top-2000-2024.xlsx...
2000 liedjes in de editie
   11 liedjes die in 2024 zijn uitgebracht
 1931 liedjes exact gematched
    3 liedjes handmatig gematched
Building similarity matrix...
Done
    4 liedjes gematched vanuit de similarity matrix
   51 liedjes niet gematched dus als nieuw toegevoegd
Edition_data:    0 +   62 + 1938 +    0 +    0 = 2000
Picklist    : 2860 + 1938 = 4798


## Werkwijze

1. Kopieer het blok daarboven en wijzig klassenaam op 2 plaatsen
1. Zet `min_certain_ratio` op 0.99 en `min_relevant_ratio` op 0.50
1. Run alle cellen tot en met de nieuwe. Er ontstaat een lijst met probleemgevallen
1. Check de probleemgevallen; begin bovenaan en verlaag `min_certain_ratio` zodat de laatst zekere ervan verdwijnt
1. Kijk vanaf onderen; Verhoog `min_relevant_ratio` totdat de laatste wel een match is
1. Er blijven nu maar een paar probleemgevallen over. Kopieer nu alle matches daaruit naar de parameter van `processor.process_manual_matches`
1. Als er geen match meer aanwezig is verhoog je `min_relevant_ratio` tot > de hoogst aanwezige ratio

# Extraheren van artiesten
Hier wordt ervoor gezorgd dat alle liedjes worden gegroepeerd per artiest.

In [19]:
artists = []

for s in t2k_data:
    # find exact match
    found = False
    for a in artists:
        if a.is_exact_match(s):
            a.add_song(s)
            found=True
            break
    # TODO find simillarities
    # add as new
    if not found:
        artists.append(Artist(s))
        
print("Extracted {} artists".format(len(artists)))

Extracted 2074 artists


# Sanity check
Check of er redelijkerwijs complete en correcte data in zit.

In [20]:
songs_with_no_year = []
for a in artists:
    for s in a.songs:
        if not hasattr(s, 'year'):
            songs_with_no_year.append(s.artist)
            
print("Aantal songs zonder jaar: {}".format(len(songs_with_no_year)))
                

Aantal songs zonder jaar: 0


In [21]:
songs_with_multiple_year = []

MAX_DIFFERENCE = 10

for a in artists:
    for s in a.songs:
        yMin = None
        yMax = None
        m = False
        for o in s.occurances:
            year = o.data[Occurance.YEAR_INDEX]
            if yMin is None:
                yMin = year
            if yMax is None:
                yMax = year
            if year < yMin:
                yMin = year
            if year > yMax:
                yMax = year
        if yMax - yMin > MAX_DIFFERENCE:
            songs_with_multiple_year.append(s)
            
print("Aantal songs met teveel spreiding in jaren: {}".format(len(songs_with_multiple_year)))
for s in songs_with_multiple_year:
    print(s)
    for o in s.occurances:
        print("{} : {}".format(o.data[Occurance.EDITION_INDEX], o))


Aantal songs met teveel spreiding in jaren: 10
André Hazes - Zij Gelooft In Mij (2004) (2004)
2003 : André Hazes - Zij Gelooft In Mij (2000): 1039
2004 : André Hazes - Zij Gelooft In Mij (2000): 30
2005 : André Hazes - Zij Gelooft In Mij (2000): 62
2006 : André Hazes - Zij Gelooft In Mij (2000): 101
2007 : André Hazes - Zij Gelooft In Mij (2000): 291
2008 : André Hazes - Zij Gelooft In Mij (2000): 123
2009 : André Hazes - Zij Gelooft In Mij (2000): 215
2010 : André Hazes - Zij Gelooft In Mij (2000): 192
2011 : André Hazes - Zij Gelooft In Mij (2000): 208
2012 : André Hazes - Zij Gelooft In Mij (2000): 169
2013 : André Hazes - Zij Gelooft In Mij (2000): 187
2014 : André Hazes - Zij Gelooft In Mij (2000): 275
2015 : André Hazes - Zij Gelooft In Mij (2000): 283
2016 : André Hazes - Zij Gelooft In Mij (2000): 256
2017 : André Hazes - Zij Gelooft In Mij (2000): 254
2018 : André Hazes - Zij Gelooft In Mij (1981): 254
2019 : André Hazes - Zij Gelooft In Mij (1981): 240
2020 : André Hazes - Zi

In [22]:
#TODO Similarities van artiestnamen per artiest
#TODO Similarities van artiestnamen binnen songs van een artiest
#TODO Similarities van titels per song

# Load: in de database opslaan
Hier wordt de data uiteindelijk in de database opgeslagen.

In [23]:
import db
db.occurrence_repository.truncate()
db.song_repository.truncate()
db.artist_repository.truncate()

artist_map = {}

for a in artists:
    a_id = db.artist_repository._insert_row([a.name])
    for s in a.songs:
        s_id = db.song_repository.append(a_id, s.title, s.year)
        for o in s.occurances:
            db.occurrence_repository.append([s_id] + o.data)
    
    
db.commit()